In [1]:
import numpy as np
import pandas as pd
import json
from konlpy.tag import Kkma, Twitter
import gensim
import re
from gensim import corpora

In [2]:
word2vecmodel = gensim.models.Word2Vec.load('./word2vecmodel')
ldamodel =  gensim.models.ldamodel.LdaModel.load('./ldamodel')
dictionary = corpora.Dictionary.load('ldamodel.id2word')
Tw = Twitter()

In [3]:
with open('con_Q_part.json','r') as fp:
    con_Q_part = json.load(fp)
with open('con_A_part.json','r') as fp:
    con_A_part = json.load(fp)
with open('pro_Q_part.json','r') as fp:
    pro_Q_part = json.load(fp)
with open('pro_A_part.json','r') as fp:
    pro_A_part = json.load(fp)
    
with open('all_Q_part.json','r') as fp:
    all_Q_part = json.load(fp)
with open('all_A_part.json','r') as fp:
    all_A_part = json.load(fp)

In [6]:
con_Q_part_vec = {}
con_A_part_vec = {}
pro_Q_part_vec = {}
pro_A_part_vec = {}

In [7]:
all_Q_part_vec = {}
all_A_part_vec = {}

# Word2vec 200차원 벡터를 리턴 해주는 함수

In [8]:
def return_word2veclist(morphs_Q, morphs_A):
    k_Q = []
    k_A = []
    
    for word in morphs_Q:
        try :
            k_Q.append(word2vecmodel.wv[word])
        except:
            k_Q.append(np.zeros(200))
            
    for word in morphs_A:
        try:
            k_A.append(word2vecmodel.wv[word])
        except:
            k_A.append(np.zeros(200))
            
    k_Q = np.array(k_Q)
    k_A = np.array(k_A)
    
    vec_Q = np.zeros(200)
    vec_A = np.zeros(200)        

    for j in range(len(vec_Q)):
        vec_Q[j] = (np.mean(k_Q[:,j]))
        vec_A[j] = (np.mean(k_A[:,j]))
    
    return(vec_Q, vec_A)

# LDA 30차원 벡터를 리턴 해주는 함수 

In [9]:
def return_ldalist(nouns_Q ,nouns_A):

    doc2bow_Q = [dictionary.doc2bow(nouns_Q)]
    doc2bow_A = [dictionary.doc2bow(nouns_A)]
    
    ldalist_Q = np.zeros(30)
    ldalist_A = np.zeros(30)
    
    if len(nouns_Q) == 0:
        ldalist_Q = np.zeros(30)
    else:
        for pair in ldamodel.get_document_topics(doc2bow_Q)[0]:
            ldalist_Q[pair[0]] = pair[1]
        
    if len(nouns_A) == 0:
        ldalist_Q = np.zeros(30)
    else:
        for pair in ldamodel.get_document_topics(doc2bow_A)[0]:
            ldalist_A[pair[0]] = pair[1]
    
    return(ldalist_Q, ldalist_A)

# 보수, 진보, 전체에 벡터들 저장

In [10]:
#convec
for key in con_Q_part.keys():
    for i in range(len(con_Q_part[key])):
        
        morphs_Q = Tw.morphs(con_Q_part[key][i])
        morphs_A = Tw.morphs(con_A_part[key][i])
        
        nouns_Q = Tw.nouns(con_Q_part[key][i])
        nouns_A = Tw.nouns(con_A_part[key][i])
        
        
        ldalist_Q, ldalist_A = return_ldalist(nouns_Q, nouns_A)
        vec_Q, vec_A = return_word2veclist(morphs_Q, morphs_A)
    
        con_Q_part_vec.setdefault(key,[])
        con_Q_part_vec[key].append(np.append(vec_Q,ldalist_Q).tolist())
        #con_Q_part_vec[key].append(vec_Q.tolist())
        
        con_A_part_vec.setdefault(key,[])
        con_A_part_vec[key].append(np.append(vec_A,ldalist_A).tolist())
        #con_A_part_vec[key].append(vec_A.tolist())
        
#provec    
for key in pro_Q_part.keys():
    for i in range(len(pro_Q_part[key])):
        
        morphs_Q = Tw.morphs(pro_Q_part[key][i])
        morphs_A = Tw.morphs(pro_A_part[key][i])
        
        nouns_Q = Tw.nouns(pro_Q_part[key][i])
        nouns_A = Tw.nouns(pro_A_part[key][i])
        
        
        ldalist_Q, ldalist_A = return_ldalist(nouns_Q, nouns_A)
        vec_Q, vec_A = return_word2veclist(morphs_Q, morphs_A)
        
        pro_Q_part_vec.setdefault(key,[])
        pro_Q_part_vec[key].append(np.append(vec_Q,ldalist_Q).tolist())
        #pro_Q_part_vec[key].append(vec_Q.tolist())
        
        pro_A_part_vec.setdefault(key,[])
        pro_A_part_vec[key].append(np.append(vec_A,ldalist_A).tolist())
        #pro_A_part_vec[key].append(vec_A.tolist())
        
#allvec
for key in all_Q_part.keys():
    for i in range(len(all_Q_part[key])):
        
        morphs_Q = Tw.morphs(all_Q_part[key][i])
        morphs_A = Tw.morphs(all_A_part[key][i])
        
        nouns_Q = Tw.nouns(all_Q_part[key][i])
        nouns_A = Tw.nouns(all_A_part[key][i])
        
        ldalist_Q, ldalist_A = return_ldalist(nouns_Q, nouns_A)
        vec_Q, vec_A = return_word2veclist(morphs_Q, morphs_A)
        
        all_Q_part_vec.setdefault(key,[])
        all_Q_part_vec[key].append(np.append(vec_Q,ldalist_Q).tolist())
        #all_Q_part_vec[key].append(vec_Q.tolist())
        
        all_A_part_vec.setdefault(key,[])
        all_A_part_vec[key].append(np.append(vec_A,ldalist_A).tolist())
        #all_A_part_vec[key].append(vec_A.tolist())

In [11]:
with open('con_Q_part_vec.json','w') as fp:
    json.dump(con_Q_part_vec,fp)
with open('con_A_part_vec.json','w') as fp:
    json.dump(con_A_part_vec,fp)
with open('pro_Q_part_vec.json','w') as fp:
    json.dump(pro_Q_part_vec,fp)
with open('pro_A_part_vec.json','w') as fp:
    json.dump(pro_A_part_vec,fp)
    
with open('all_Q_part_vec.json','w') as fp:
    json.dump(all_Q_part_vec,fp)
with open('all_A_part_vec.json','w') as fp:
    json.dump(all_A_part_vec,fp)